# Master Data Science Data_VIZ
### WADE El Hadji Malick
### HAYKAL Fayad

## 1. Informations sur les données 

In [1]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

path = "/users/mmath/wade/Bureau/Master_DS/Projets/Data_viz/Synthese_donnees_des_semis.csv"
df = pd.read_csv(path,  sep=';')

df.head()

,Bancs,camera,zone,Pop,5_degres_C_TMG_h,5_degres_C_TMG_j,Aire_sous_la_courbe,15_j,16 _j,17_j,18 _j,19_j,20 _j,21_j
0,3.0,1.0,11.0,Témoin,"414,00","17,25",2652,4.0,16.0,20.0,24.0,32.0,36.0,40.0
1,3.0,2.0,11.0,Témoin,"430,86","17,95",2652,0.0,4.0,4.0,12.0,24.0,28.0,28.0
2,3.0,3.0,11.0,Témoin,"382,86","15,95",2652,8.0,12.0,20.0,20.0,20.0,24.0,28.0
3,3.0,4.0,11.0,Témoin,"420,00","17,50",2652,4.0,4.0,4.0,4.0,12.0,16.0,16.0
4,4.0,1.0,11.0,Témoin,"432,89","18,04",2646,4.0,4.0,16.0,16.0,24.0,32.0,36.0


In [3]:
df.dtypes
df[['POP','5_degres_C_TMG_h', '5_degres_C_TMG_j', 'Aire_sous_la_courbe']] = df[['POP','5_degres_C_TMG_h', '5_degres_C_TMG_j', 'Aire_sous_la_courbe']].astype(float)

KeyError: "['POP'] not in index"

In [ ]:
import seaborn as sns
sns.set(style="ticks", color_codes=True)

g = sns.pairplot(df)